# Install and run the cuOpt server

## Load notebooks from github

In [ ]:
%%bash
mkdir examples
git clone  -b branch-25.02 --single-branch https://github.com/NVIDIA/cuopt-examples
mv cuopt-examples/LICENSE.md cuopt-examples/README.md ./examples 
for d in cuFOLIO_portfolio_optimization sample_lp_sever_notebooks; do
    mv cuopt-examples/$d ./examples
    if [ -f ./examples/$d/requirements.txt ]; then
        pip install -r ./examples/$d/requirements.txt
    fi
done
rm -rf cuopt-examples
echo
echo Please refresh the file navigator pane.

## Install the cuOpt self-hosted client

In [ ]:
%%bash
pip install cuopt_sh_client --extra-index-url=https://pypi.nvidia.com

## Add the current user to the docker group

#### This requires passwordless sudo and allows the current user to run containers

In [ ]:
%%bash
sudo usermod -aG docker $USER
sudo su $USER

## Enter a NGC api key and download the cuOpt container image
### Note: you must advance manually to the next cell after successfully setting an api key

In [ ]:
import getpass, os
os.environ["CUOPT_IMAGE"] = "nvcr.io/nvidia/cuopt/cuopt-ea:25.02"
if os.environ.get("NGC_API_KEY"):
    p = getpass.getpass("You have an api key set already. Enter a new api key or simply press enter to keep the current key: ")
    if p:
        os.environ["NGC_API_KEY"] = p
        print("Successfully set api key.")        
    else:
        print("Current api key kept")
else:
    os.environ["NGC_API_KEY"] = getpass.getpass("Enter your api key and press enter: ")
    if os.environ["NGC_API_KEY"]:
        print("Successfully set api key.")

In [ ]:
%%bash
docker logout nvcr.io
echo "${NGC_API_KEY}" | docker login nvcr.io -u '$oauthtoken' --password-stdin
echo pulling image ....
docker pull $CUOPT_IMAGE
docker logout nvcr.io

In [ ]:
del os.environ['NGC_API_KEY']

## Start the cuOpt server

In [ ]:
import subprocess
import os

# Make sure we do not already have the cuopt server running
docker_cmd = [
    "docker", "ps"
]
ps = subprocess.run(docker_cmd, capture_output=True, text=True)
for l in ps.stdout.split("\n"):
    if "cuopt" in l:
        container_id = l.split()[0]
        print("cuopt is already running.")    
        print(container_id)
        break
else:
    docker_cmd = [
        "docker", "run",
        "--rm", "-d",
        "--network=host",
        "--gpus=1", "-it",
        os.environ['CUOPT_IMAGE']
    ]
    container_id = subprocess.run(docker_cmd, capture_output=True, text=True).stdout.strip()
    print("cuopt started")
    print(container_id)



## Stop the cuOpt server

In [ ]:
import subprocess
import os

docker_cmd = [
    "docker", "stop", container_id
]
ps = subprocess.run(docker_cmd, capture_output=True, text=True)
if ps.stderr:
    print(ps.stderr)
else:
    print("cuopt server stopped.")

### COPYRIGHT and LICENSE

SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
